The following code takes the cell count matrix and a metadata file to generate an h5ad file containing annotated data which will be used for the downstream analysis

The commands might require anywhere around 50GB of memory. Another option to a high spec machine is using Google Colab with a TPU runtime, which provides 335GB of memory for free.

We will start by importing some libraries. The function of each library will be explained in the following code blocks

In [5]:
import os
import numpy as np
import pandas as pd
import scipy
import anndata
import scanpy as sc
import seaborn as sns

In [7]:
# Working with relative directories is the best way to ensure that your code is reproducible
cwd = os.getcwd()

# Load the metadata using pandas dataframes
meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t')
meta.drop([0], axis=0, inplace=True)
meta.rename(columns={'NAME':'CellID', 'Cell_line':'CellLine', 'Pool_ID':'Pool', 'Cancer_type':'Indication'}, inplace=True)
meta

<ipython-input-7-f5fdd852fb4d>:2: DtypeWarning: Columns (4,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv(cwd+'/data/Metadata.txt', sep='\t')


,CellID,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,...,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
1,AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,NaN,NaN,NaN,NaN,0.166,...,-0.935,-0.935,0.13,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
2,AACGTTGTCACCCGAG-1-18,NCIH2126_LUNG,18,Lung Cancer,5200,NaN,NaN,NaN,NaN,-0.213,...,-1.027,-1.027,0.066,1.049,1.267,0.252,1.299,1.61,0.624,-0.048
3,AACTGGTAGACACGAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4004,NaN,NaN,NaN,NaN,-0.101,...,-0.677,-0.677,0.304,0.822,2.401,0.141,0.451,1.225,-0.795,0.064
4,AACTGGTAGGGCTTGA-1-18,NCIH2126_LUNG,18,Lung Cancer,4295,NaN,NaN,NaN,NaN,-0.014,...,-0.735,-0.735,0.094,0.834,2.282,0.15,0.267,0.892,-0.238,1.118
5,AACTGGTAGTACTTGC-1-18,NCIH2126_LUNG,18,Lung Cancer,4842,NaN,NaN,NaN,NaN,0.006,...,-0.821,-0.821,0.034,0.96,1.4,-0.012,-0.276,-0.428,0.267,0.791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53509,c4722,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3343,NaN,NaN,NaN,NaN,0.018,...,-0.505,-0.505,1.657,1.583,3.85,0.539,0.473,0.544,-1.079,-1.349
53510,c4724,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6977,NaN,NaN,NaN,NaN,-0.098,...,-0.876,-0.876,0.669,1.086,3.046,0.799,0.49,1.319,-0.37,0.057
53511,c4731,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,6638,NaN,NaN,NaN,NaN,-0.112,...,-0.112,-0.112,0.61,0.693,2.289,0.65,0.729,1.143,-0.508,0.501
53512,c4735,JHU006_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4052,NaN,NaN,NaN,NaN,-0.244,...,1.981,1.981,0.523,-0.309,0.267,0.822,1.049,0.777,0.296,-0.936


Reading the cell IDs to add them to the final annotated file

In [12]:
counts_cellid = pd.read_csv(cwd+'/data/UMIcount_data.txt', nrows=1, sep='\t', header=None)
counts_cellid = counts_cellid.transpose()
counts_cellid.drop([0], inplace=True)
counts_cellid

,0
1,AAACCTGAGACATAAC-1-18
2,AAACCTGCACAACGCC-1-18
3,AAACCTGCAGACAAGC-1-18
4,AAACCTGCAGCTCGAC-1-18
5,AAACCTGCATGGATGG-1-18
...,...
56978,c4788
56979,c4789
56980,c4793
56981,c4800


The next code blocks might abuse your memory, check he available memory first with psutil

In [ ]:
import psutil

# Check the available memory
print(f"Available memory: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Reading counts data into a pd dataframe

In [10]:
%%time
counts = pd.read_csv(cwd+'/data/UMIcount_data.txt', sep='\t', skiprows=3, header=None, index_col=0)
counts = counts.transpose() # Transpose the counts matrix to have cells as rows and genes as columns
counts

CPU times: user 11min 44s, sys: 42 s, total: 12min 26s
Wall time: 12min 13s


,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,...,PRICKLE4,RABL6,RAET1E-AS1,RGS5,SERPINA3,SPATA13,TBC1D26,TIMM10B,TMBIM4,TMEM256-PLSCR3
1,0,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56978,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,4,4,0
56979,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
56980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
56981,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# Giving index to the counts matrix to match the cellID
counts.index = counts_cellid[0]

# Remove the index name to avoid conflicts with the metadata
counts.index.name = None

# Filter the counts matrix to keep only the cells that are present in the metadata
a = counts.index.isin(meta['CellID'])
counts = counts[a]
counts

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A2ML1-AS2,A3GALT2,A4GALT,...,PRICKLE4,RABL6,RAET1E-AS1,RGS5,SERPINA3,SPATA13,TBC1D26,TIMM10B,TMBIM4,TMEM256-PLSCR3
AAACCTGAGACATAAC-1-18,0,0,0,0,0,0,0,0,0,0,...,0,6,0,0,0,0,0,1,0,0
AAACCTGCACAACGCC-1-18,1,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
AAACCTGCAGACAAGC-1-18,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,1,0,0,0,0
AAACCTGCAGCTCGAC-1-18,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCTGCATGGATGG-1-18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,1,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,4,4,0
c4789,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
c4793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
c4800,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
# Set the index of the metadata to be the CellID to match the counts matrix
meta = meta.set_index('CellID')

# Reorder the metadata to match the counts matrix order, this is important for downstream analysis
meta = meta.reindex(index=counts.index)
meta

,CellLine,Pool,Indication,Genes_expressed,Discrete_cluster_minpts5_eps1.8,Discrete_cluster_minpts5_eps1.5,Discrete_cluster_minpts5_eps1.2,CNA_subclone,SkinPig_score,EMTI_score,EMTII_score,EMTIII_score,IFNResp_score,p53Sen_score,EpiSen_score,StressResp_score,ProtMatu_score,ProtDegra_score,G1/S_score,G2/M_score
AAACCTGAGACATAAC-1-18,NCIH2126_LUNG,18,Lung Cancer,4318,NaN,NaN,NaN,NaN,0.166,-0.045,-0.935,-0.935,0.13,0.619,1.869,-0.004,0.805,0.896,0.424,-1.125
AAACCTGCACAACGCC-1-18,SW579_THYROID,18,Thyroid Cancer,5021,NaN,NaN,SW579_THYROID_1,NaN,-0.056,0.776,0.953,0.953,-0.266,-0.334,-1.125,-0.039,-0.243,-0.642,-0.173,1.365
AAACCTGCAGACAAGC-1-18,C32_SKIN,18,Skin Cancer,3047,NaN,NaN,NaN,NaN,1.092,0.617,-0.034,-0.034,0.318,0.57,-0.165,0.074,0.25,0.096,-0.367,-1.135
AAACCTGCAGCTCGAC-1-18,SW579_THYROID,18,Thyroid Cancer,2765,NaN,NaN,SW579_THYROID_1,NaN,-0.601,1.038,1.952,1.952,0.341,-0.253,-0.552,0.921,2.876,1.645,0.226,0.469
AAACCTGCATGGATGG-1-18,NCIH446_LUNG,18,Lung Cancer,2064,NaN,NaN,NaN,NaN,-0.251,-0.325,0.258,0.258,-0.044,-1.256,-0.367,-0.317,0.79,1.925,0.138,-0.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c4788,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,5929,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_1,NaN,-0.317,-0.39,0.023,0.023,-0.1,-0.604,-0.358,-0.123,0.067,0.804,0.135,0.264
c4789,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,3531,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,SCC9_UPPER_AERODIGESTIVE_TRACT_2,0.238,1.176,2.094,2.094,0.014,-0.045,-0.05,0.214,0.238,-0.514,-1.021,-1.243
c4793,JHU029_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4029,JHU029_UPPER_AERODIGESTIVE_TRACT_1,JHU029_UPPER_AERODIGESTIVE_TRACT_3,NaN,NaN,-0.164,-0.163,-0.405,-0.405,0.01,-0.008,0.045,0.265,0.054,-0.466,0.345,0.804
c4800,SCC9_UPPER_AERODIGESTIVE_TRACT,custom,Head and Neck Cancer,4009,SCC9_UPPER_AERODIGESTIVE_TRACT_1,SCC9_UPPER_AERODIGESTIVE_TRACT_1,SCC9_UPPER_AERODIGESTIVE_TRACT_4,SCC9_UPPER_AERODIGESTIVE_TRACT_1,0.205,0.217,0.833,0.833,-0.257,-0.856,-0.593,0.152,0.022,0.684,0.684,0.624


In [16]:
# Finally, we check if the index of the metadata and the counts matrix are the same
counts.index.equals(meta.index)

True

In [17]:
# Finally, we can create an AnnData object to store the counts matrix and the metadata
# AnnData objects are the standard way to store single-cell data in scanpy and other single-cell analysis tools
adata = anndata.AnnData(X=scipy.sparse.csr_matrix(counts),
                        obs=meta,
                        var=counts.columns.to_frame())
del counts
adata.var.drop(columns=[0], inplace=True)
adata.var.index.name = None
adata

AnnData object with n_obs × n_vars = 53513 × 30314
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score'

In [19]:
# Define the columns and their respective target data types
column_types = {
    'CellLine': str,
    'Pool': str,
    'Indication': str,
    'Genes_expressed': int,
    'Discrete_cluster_minpts5_eps1.8': str,
    'Discrete_cluster_minpts5_eps1.5': str,
    'Discrete_cluster_minpts5_eps1.2': str,
    'CNA_subclone': str,
    'SkinPig_score': float,
    'EMTI_score': float,
    'EMTII_score': float,
    'EMTIII_score': float,
    'IFNResp_score': float,
    'p53Sen_score': float,
    'EpiSen_score': float,
    'StressResp_score': float,
    'ProtMatu_score': float,
    'ProtDegra_score': float,
    'G1/S_score': float,
    'G2/M_score': float
}

# Apply the data types to the columns
for col, dtype in column_types.items():
    adata.obs[col] = adata.obs[col].astype(dtype)


In [20]:
# Using the scanpy function to filter genes and cells based on the number of cells expressing a gene and the number of genes expressed by a cell
# Scanpy is a powerful tool for single-cell analysis and provides many functions to preprocess and analyze single-cell data
sc.pp.filter_genes(adata, min_cells=10)
sc.pp.filter_cells(adata, min_genes=200)
adata

AnnData object with n_obs × n_vars = 53513 × 23081
    obs: 'CellLine', 'Pool', 'Indication', 'Genes_expressed', 'Discrete_cluster_minpts5_eps1.8', 'Discrete_cluster_minpts5_eps1.5', 'Discrete_cluster_minpts5_eps1.2', 'CNA_subclone', 'SkinPig_score', 'EMTI_score', 'EMTII_score', 'EMTIII_score', 'IFNResp_score', 'p53Sen_score', 'EpiSen_score', 'StressResp_score', 'ProtMatu_score', 'ProtDegra_score', 'G1/S_score', 'G2/M_score', 'n_genes'
    var: 'n_cells'

In [22]:
# Writing the AnnData object to a file that can be read in the downstream analysis steps
adata.write(cwd+'/outs/240701_kinker_anndata.h5ad')